In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu117


In [2]:
%pip install transformers requests beatifulsoup4 pandas numpy

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement beatifulsoup4 (from versions: none)
ERROR: No matching distribution found for beatifulsoup4

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install html5lib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

d:\Software\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
from urllib.request import urlopen
url = "https://us.trip.com/things-to-do/detail/16027544/?_gl=1*eqgzyw*_gcl_aw*R0NMLjE2ODMzNzYxMjYuQ2owS0NRanc5ZGVpQmhDMUFSSXNBSExqUjJBX2pNUW5OSVFBclhLQTZFRXhvWHpPNHRNMTlCM3puakh0OVNQbVNobWdFYWF4V2hrWUo1a2FBcXd3RUFMd193Y0I.*_gcl_dc*R0NMLjE2ODMzNzYxMjYuQ2owS0NRanc5ZGVpQmhDMUFSSXNBSExqUjJBX2pNUW5OSVFBclhLQTZFRXhvWHpPNHRNMTlCM3puakh0OVNQbVNobWdFYWF4V2hrWUo1a2FBcXd3RUFMd193Y0I." # replace with the URL of the website you want to scrape data from
html = urlopen(url)

In [4]:

soup = BeautifulSoup(html, 'lxml')


In [5]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [6]:
# Set up the Selenium driver
driver = webdriver.Firefox()

# Navigate to the page you want to scrape
url = "https://www.expedia.com/things-to-do/budapest-danube-river-sightseeing-night-cruise-by-legenda-city-cruises.a846150.activity-details?endDate=2023-05-21&location=Budapest%20%28and%20vicinity%29%2C%20Hungary&rid=179994&selectedId=&sort=RECOMMENDED&startDate=2023-05-20&swp=on"
driver.get(url)

# Wait for the page to load
driver.implicitly_wait(10)

# Get the page's HTML content
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'lxml')



In [7]:
# Find the comments on the page
comments = soup.find_all('div', class_='uitk-text overflow-wrap uitk-type-300 uitk-text-default-theme')

# Extract and print the text of each comment
for comment in comments:
    comment_text = comment.get_text()
    print(comment_text)

Stunning city, loved it, happy to recommend, hope we can return. Night cruise was amazing. Plenty of time for photos.
Peccato che l audio guida non funzionava. Per il resto tutto ok. Nave silenziosa e pulita. Buona organizzazione.
Very good friendly service from the staff, very attentive and helpful. The audio tour was very good and informative and overall value for money was excellent.
Tolles Schiff mit guter Rundumsicht, sehr leise und sehr gutem Audioguide. Pünktliche Abfahrt, bequeme Sitze, alles in Ordnung!
Lovely to see the city by night with all the lights. Well organised trip. The headphones with a range of languages was helpful and gavelots of interesting information.


In [8]:
# Find the comments on the page
comments = soup.find_all('div', class_='uitk-text overflow-wrap uitk-type-300 uitk-text-default-theme')

# Create an empty list to store the comments
comments_list = []

# Extract and append the text of each comment to the list
for comment in comments:
    comment_text = comment.get_text()
    comments_list.append(comment_text)
print(comments_list[1])

# Now you have a list of comments that you can use for sentiment analysis

Peccato che l audio guida non funzionava. Per il resto tutto ok. Nave silenziosa e pulita. Buona organizzazione.


In [9]:
tokens=tokenizer.encode(comment_text, return_tensors="pt")

result = model(tokens)

result.logits

int(torch.argmax(result.logits))+1
print(comment_text)

Lovely to see the city by night with all the lights. Well organised trip. The headphones with a range of languages was helpful and gavelots of interesting information.


In [10]:
# Iterate over the comments in the comments_list array
for comment_text in comments_list:
    # Tokenize the comment_text
    tokens = tokenizer.encode(comment_text, return_tensors="pt")

    # Pass the tokens to the model
    result = model(tokens)

    # Get the predicted sentiment from the model's output logits
    sentiment = int(torch.argmax(result.logits)) + 1

    # Print the comment and its predicted sentiment
    print(f"Comment: {comment_text}")
    print(f"Predicted sentiment: {sentiment}")

Comment: Stunning city, loved it, happy to recommend, hope we can return. Night cruise was amazing. Plenty of time for photos.
Predicted sentiment: 5
Comment: Peccato che l audio guida non funzionava. Per il resto tutto ok. Nave silenziosa e pulita. Buona organizzazione.
Predicted sentiment: 3
Comment: Very good friendly service from the staff, very attentive and helpful. The audio tour was very good and informative and overall value for money was excellent.
Predicted sentiment: 5
Comment: Tolles Schiff mit guter Rundumsicht, sehr leise und sehr gutem Audioguide. Pünktliche Abfahrt, bequeme Sitze, alles in Ordnung!
Predicted sentiment: 5
Comment: Lovely to see the city by night with all the lights. Well organised trip. The headphones with a range of languages was helpful and gavelots of interesting information.
Predicted sentiment: 5


In [26]:
# Find the user-rated values on the page
user_scores = soup.find_all('span', class_='is-visually-hidden')

# Create an empty list to store the user-rated values
user_scores_list = []

# Extract and append the first character of each user score to the list, skipping the first instance
for score in user_scores[-5:]:
    score_text = score.get_text()
    if score_text:
        try:
            user_score = float(score_text[0])
            user_scores_list.append(user_score)
        except ValueError:
            # The first character could not be converted to a float, so skip it
            pass
# Iterate over the user scores in the user_scores_list array
for user_score in user_scores_list:
    # Print the user score
    print(user_score)

5.0
4.0
5.0
4.0
5.0


In [27]:
# Iterate over the comments and user scores
for comment_text, user_score in zip(comments_list, user_scores_list):
    # Tokenize the comment_text
    tokens = tokenizer.encode(comment_text, return_tensors="pt")

    # Pass the tokens to the model
    result = model(tokens)

    # Get the predicted sentiment from the model's output logits
    sentiment = int(torch.argmax(result.logits)) + 1

    # Print the comment, its predicted sentiment, and the user-rated value
    print(f"Comment: {comment_text}")
    print(f"Predicted sentiment: {sentiment}")
    print(f"User-rated value: {user_score}")

    # Compare the predicted sentiment with the user-rated value
    if sentiment == user_score:
        print("The predicted sentiment matches the user-rated value")
    else:
        print("The predicted sentiment does not match the user-rated value")

Comment: Stunning city, loved it, happy to recommend, hope we can return. Night cruise was amazing. Plenty of time for photos.
Predicted sentiment: 5
User-rated value: 5.0
The predicted sentiment matches the user-rated value
Comment: Peccato che l audio guida non funzionava. Per il resto tutto ok. Nave silenziosa e pulita. Buona organizzazione.
Predicted sentiment: 3
User-rated value: 4.0
The predicted sentiment does not match the user-rated value
Comment: Very good friendly service from the staff, very attentive and helpful. The audio tour was very good and informative and overall value for money was excellent.
Predicted sentiment: 5
User-rated value: 5.0
The predicted sentiment matches the user-rated value
Comment: Tolles Schiff mit guter Rundumsicht, sehr leise und sehr gutem Audioguide. Pünktliche Abfahrt, bequeme Sitze, alles in Ordnung!
Predicted sentiment: 5
User-rated value: 4.0
The predicted sentiment does not match the user-rated value
Comment: Lovely to see the city by night